In [214]:
# ! jupyter nbconvert --to python --RegexRemovePreprocessor.patterns="^%"  train_and_run.ipynb

# RULES:
# 1. Do not make plot.show() only plot.savefig()

In [1]:
import socket
print(socket.gethostname())
import sys
sys.path.append("../../fande")

info01


In [2]:
from fande.utils.find_atomic_groups import find_atomic_groups

fande.models module imported...


In [3]:
find_atomic_groups("/home/qklmn/data/starting_configuration/triazine/295optdftb.cif")

Total number of found groups: 14
Checking if all atoms are covered:  True


[[0, 1, 2, 3, 4, 5, 264, 265, 266, 267, 268, 269],
 [146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,
  155,
  156,
  157,
  158,
  159,
  160,
  161,
  162,
  163,
  168,
  169,
  170,
  171,
  172,
  173,
  174,
  175,
  176,
  177,
  178,
  179,
  180,
  181,
  182,
  183,
  184,
  185,
  186,
  187,
  188,
  189,
  190,
  191,
  194,
  195,
  196,
  197,
  198,
  199,
  200,
  201,
  202,
  203,
  204,
  205,
  208,
  209,
  210,
  211,
  212,
  213,
  214,
  215,
  216,
  217,
  218,
  219,
  220,
  221,
  222,
  223,
  224,
  225,
  226,
  227,
  228,
  229,
  230,
  231,
  232,
  233,
  234,
  235,
  236,
  237,
  238,
  239,
  240,
  241,
  242,
  243,
  244,
  245,
  246,
  247,
  248,
  249,
  252,
  253,
  254,
  255,
  256,
  257,
  258,
  259,
  260,
  261,
  262,
  263,
  410,
  411,
  412,
  413,
  414,
  415,
  416,
  417,
  418,
  419,
  420,
  421,
  422,
  423,
  424,
  425,
  426,
  427,
  432,
  433,
  434,
  435,
  436,
  437,
  438,
  439,
  440,
  44

In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
from xtb.ase.calculator import XTB
from ase import io
from ase.build import molecule

atoms_copy = io.read( os.path.expanduser("/home/qklmn/data/starting_configuration/triazine/295optdftb.cif"), format="cif" )#molecule('H2O')

atoms_copy.set_pbc(False)
calc_xtb = XTB(method='GFN2-xTB')
atoms_copy.set_calculator(calc_xtb)

atoms_copy.get_potential_energy()

NameError: name 'io' is not defined

## Make sweep

In [1]:
# # https://docs.wandb.ai/guides/sweeps/walkthrough

import os
machine_name = os.uname()[1]
import wandb
wandb.init(project="rotor-gp", save_code=True, notes="hello", id=machine_name, mode='disabled')

# sweep_configuration = {
#     'method': 'random',
#     'metric': 
#     {
#         'goal': 'minimize', 
#         'name': 'score'
#         },
#     'parameters': 
#     {
#         'x': {'max': 0.1, 'min': 0.01},
#         'y': {'values': [1, 3, 7]},
#      }
# }

## Data loading part

In [2]:
from prepare_model import prepare_data, sample_data, prepare_model


In [100]:
# Hyperparameters:
hparams = {
        'dtype' : 'float32',
        'device' : 'gpu'
        }

# Descriptors parameters:
# https://github.com/lab-cosmo/librascal/blob/master/examples/MLIP_example.ipynb
soap_params = {
# 'species': ["H", "C", "O", "N", "Si"],
# 'periodic': True,
'interaction_cutoff': 3.5,
'gaussian_sigma_constant': 0.3,
'max_radial': 4,
'max_angular': 4,
'cutoff_smooth_width': 0.1,
# 'average': "off",
# 'crossover': True,
# 'dtype': "float64",
# 'n_jobs': 10,
# 'sparse': False,
# 'positions': [7, 11, 15] # ignored
}


fdm = prepare_data(hparams, soap_params, traj_sample_rate=1)


Saving data to directory:  /misc/home/qklmn/repos/rotor-gp/results/train_and_run/2023-08-25_16 52-26_952780
Loading training data...
Length of the train trajectory:  400
Total number of frames is 400
Total number of batches is 400
Total length of traj is 400
Total number of batches 400
Calculating invariants on trajectory with librascal...


100%|██████████| 400/400 [02:21<00:00,  2.84it/s]


Total number of frames is 20
Total number of batches is 20
Total length of traj is 20
Total number of batches 20
Calculating invariants on trajectory with librascal...


100%|██████████| 20/20 [00:05<00:00,  3.46it/s]



-----------
Group  0
-----------
Number of training points for group 0:  14400
Number of test points for group 0:  720

-----------
Group  1
-----------
Number of training points for group 1:  14400
Number of test points for group 1:  720

-----------
Group  2
-----------
Number of training points for group 2:  21600
Number of test points for group 2:  1080

-----------
Group  3
-----------
Number of training points for group 3:  129600
Number of test points for group 3:  6480

-----------
Group  4
-----------
Number of training points for group 4:  7200
Number of test points for group 4:  360

-----------
Group  5
-----------
Number of training points for group 5:  7200
Number of test points for group 5:  360

-----------
Group  6
-----------
Number of training points for group 6:  14400
Number of test points for group 6:  720

-----------
Group  7
-----------
Number of training points for group 7:  14400
Number of test points for group 7:  720

-----------
Group  8
-----------
Numbe

In [106]:
#[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 

train_data_loaders = sample_data(fdm, N_samples=6000)

Dataloader for group 0 created
Number of samples in dataloader: 6000
Dataloader for group 1 created
Number of samples in dataloader: 6000
Dataloader for group 2 created
Number of samples in dataloader: 6000
Dataloader for group 3 created
Number of samples in dataloader: 6000
Dataloader for group 4 created
Number of samples in dataloader: 6000
Dataloader for group 5 created
Number of samples in dataloader: 6000
Dataloader for group 6 created
Number of samples in dataloader: 6000
Dataloader for group 7 created
Number of samples in dataloader: 6000
Dataloader for group 8 created
Number of samples in dataloader: 6000
Dataloader for group 9 created
Number of samples in dataloader: 6000
Dataloader for group 10 created
Number of samples in dataloader: 6000
Dataloader for group 11 created
Number of samples in dataloader: 6000
Dataloader for group 12 created
Number of samples in dataloader: 6000
Dataloader for group 13 created
Number of samples in dataloader: 6000


## Training part

In [103]:
os.getcwd()

'/misc/home/qklmn/repos/rotor-gp/results/train_and_run/2023-08-25_16 52-26_952780'

In [4]:
hparams['dtype'] = 'float32'
hparams['device'] = 'gpu'

# per AG steps and learning rate
# n_steps = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 250] 
n_steps = 250
learning_rate = [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]

AG_force_model = prepare_model(
    train_data_loaders, 
    hparams, soap_params, 
    n_steps, 
    learning_rate, 
    gpu_id=0)


### TESTING PREDICITONS ###

# os.makedirs("")

from fande.predict import PredictorASE


AG_force_model.eval()

predictor = PredictorASE(
            fdm,
            AG_force_model,
            hparams,
            soap_params
)

rmse_per_model, mae_per_model = predictor.test_errors(view_worst_atoms=True)

NameError: name 'hparams' is not defined

In [108]:
print(*rmse_per_model)

0.12341609 0.10553048 0.091876306 0.13395749 0.25830644 0.26118183 0.23828773 0.24130757 0.2764311 0.20859042 0.20170158 0.27807495 0.22754745 0.30135468


In [105]:
print(*rmse_per_model)

0.12650283 0.111152686 0.0937499 0.122396745 0.25456166 0.25662658 0.25729108 0.26040304 0.28463173 0.23395911 0.21419917 0.26237476 0.22937822 0.30425292


## Testing part

In [91]:
### TESTING PREDICITONS ###

# os.makedirs("")

from fande.predict import PredictorASE


AG_force_model.eval()

predictor = PredictorASE(
            fdm,
            AG_force_model,
            hparams,
            soap_params
)

rmse_per_model, mae_per_model = predictor.test_errors(view_worst_atoms=True)

# rmse_per_model

Error metrics for atomic group  0
RMSE:  0.20907463
MAE:  0.16433795
Max error:  0.6137252
Error metrics for atomic group  1
RMSE:  0.17848754
MAE:  0.14361839
Max error:  0.45208207
Error metrics for atomic group  2
RMSE:  0.18250507
MAE:  0.13861816
Max error:  0.6966752
Error metrics for atomic group  3
RMSE:  0.2100161
MAE:  0.16502292
Max error:  0.8838644
Error metrics for atomic group  4
RMSE:  0.35674223
MAE:  0.2941074
Max error:  0.9313594
Error metrics for atomic group  5
RMSE:  0.32867983
MAE:  0.27106744
Max error:  0.81622946
Error metrics for atomic group  6
RMSE:  0.4375405
MAE:  0.3306015
Max error:  1.6219043
Error metrics for atomic group  7
RMSE:  0.49573088
MAE:  0.39131615
Max error:  1.8473008
Error metrics for atomic group  8
RMSE:  0.4500946
MAE:  0.37787527
Max error:  1.1081257
Error metrics for atomic group  9
RMSE:  0.4421869
MAE:  0.33996886
Max error:  1.6278455
Error metrics for atomic group  10
RMSE:  0.5217358
MAE:  0.40904948
Max error:  1.7572368
Err

In [ ]:
from prepare_model import prepare_fande_ase_calc

# Hyperparameters:
hparams = {
        'dtype' : 'float32',
        'device' : 'gpu'
        }

# Descriptors parameters:
# https://github.com/lab-cosmo/librascal/blob/master/examples/MLIP_example.ipynb
soap_params = {
# 'species': ["H", "C", "O", "N", "Si"],
# 'periodic': True,
'interaction_cutoff': 3.0,
'gaussian_sigma_constant': 0.3,
'max_radial': 4,
'max_angular': 4,
'cutoff_smooth_width': 0.1,
# 'average': "off",
# 'crossover': True,
# 'dtype': "float64",
# 'n_jobs': 10,
# 'sparse': False,
# 'positions': [7, 11, 15] # ignored
}


fande_calc = prepare_fande_ase_calc(hparams, soap_params)

In [136]:
# from fande.predict import PredictorASE

# from fande.ase import FandeCalc



# predictor = PredictorASE(
#             fdm,
#             model_e,
#             trainer_e,
#             AG_force_model,
#             # trainer_f,
#             hparams,
#             soap_params
# )

# atoms = traj_test[14].copy()

# # predictor.predict_forces_single_snapshot_r(atoms)

# atoms = traj_test[5].copy()

# atoms.calc = FandeCalc(predictor)

# atoms.get_forces()

In [ ]:
# from matplotlib import pyplot as plt



# traj_1 = traj_300

# traj_2 = traj_1500

# mol = traj_300[200].copy()



# uncertainties_0 = []
# traj_0 = []
# for i in range(0,50):
#     traj_0.append(mol.copy())
#     uncertainties_0.append(FandeCalc(predictor).get_forces_variance(mol.copy())[13,1])
#     mol.positions[13,1] = mol.positions[13,1] + 0.1

# uncertainties_1 = []
# for mol in traj_1[300:320]:
#     uncertainties_1.append(FandeCalc(predictor).get_forces_variance(mol.copy())[212,0])

# uncertainties_2 = []
# for mol in traj_2[300:320]:
#     uncertainties_2.append(FandeCalc(predictor).get_forces_variance(mol.copy())[212,0])

In [ ]:
atoms.get_forces_variance()

In [223]:
# from ase import Atoms
# class AtomsWrapped(Atoms):   
#     def __init__(self, *args, **kwargs):
#         super(AtomsWrapped, self).__init__(*args, **kwargs)      
#         self.calc_history_counter = 0
    
#     def get_forces_variance(self):
#         forces_variance = super(AtomsWrapped, self).calc.get_forces_variance(self)
#         return forces_variance


#     def get_forces(self):       
#         forces = super(AtomsWrapped, self).get_forces()
#         forces_variance = super(AtomsWrapped, self).calc.get_forces_variance(self)
#         self.arrays['forces_variance'] = forces_variance
#         # energy = super(AtomsWrapped, self).get_potential_energy()
#         from ase.io import write
#         import os
#         os.makedirs("ase_calc_history" , exist_ok=True)
#         write( "ase_calc_history/" + str(self.calc_history_counter) + ".xyz", self, format="extxyz")
#         # self.calc_history.append(self.copy())       
#         self.calc_history_counter += 1
#         return forces


from fande.ase import FandeAtomsWrapper 
    
mol = io.read("/home/qklmn/data/starting_configuration/1.cif")

mol = FandeAtomsWrapper(mol)
mol.calc = FandeCalc(predictor)

In [226]:
os.getcwd()

'/misc/home/qklmn'

In [225]:
mol.get_forces_variance()

/misc/home/qklmn/repos/rotor-gp/code/../../fande/fande/predict/predictors_ase.py:571: UserWarning:

Calculating invariants...



Total number of frames is 1
Total number of batches is 1
Total length of traj is 1
Total number of batches 1
Calculating invariants on trajectory with librascal...


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  2.99it/s]
/misc/home/qklmn/repos/rotor-gp/code/../../fande/fande/predict/predictors_ase.py:595: UserWarning:

Predicting...



array([[0.28246722, 0.2569364 , 0.17697105],
       [0.16083393, 0.22986922, 0.22655949],
       [0.26595339, 0.19469914, 0.15795645],
       [0.21528253, 0.12407764, 0.21884617],
       [0.59140593, 0.39451155, 0.44077215],
       [0.43508896, 0.30540356, 0.42349371],
       [0.74045306, 0.78981966, 0.77597141],
       [0.73075187, 0.74705845, 0.75709736],
       [0.74937946, 0.74908561, 0.77944779],
       [0.75806224, 0.73596281, 0.72551358],
       [0.73399311, 0.75456107, 0.74840587],
       [0.71749824, 0.728517  , 0.7514413 ],
       [0.73656768, 0.74442554, 0.72919828],
       [0.73354727, 0.75287962, 0.7241143 ],
       [0.73433924, 0.74929017, 0.83233702],
       [0.76254344, 0.75875998, 0.77559745],
       [0.73451585, 0.77248311, 0.79197532],
       [0.74167603, 0.77837813, 0.76843727],
       [0.70716739, 0.72476864, 0.71955955],
       [0.7083602 , 0.72318393, 0.71659815],
       [0.74912465, 0.76287735, 0.77348232],
       [0.73211652, 0.74619669, 0.76579893],
       [0.

In [165]:
mol.get_forces_variance()

array([[0.28246722, 0.2569364 , 0.17697105],
       [0.16083393, 0.22986922, 0.22655949],
       [0.26595339, 0.19469914, 0.15795645],
       [0.21528253, 0.12407764, 0.21884617],
       [0.59140593, 0.39451155, 0.44077215],
       [0.43508896, 0.30540356, 0.42349371],
       [0.74045306, 0.78981966, 0.77597141],
       [0.73075187, 0.74705845, 0.75709736],
       [0.74937946, 0.74908561, 0.77944779],
       [0.75806224, 0.73596281, 0.72551358],
       [0.73399311, 0.75456107, 0.74840587],
       [0.71749824, 0.728517  , 0.7514413 ],
       [0.73656768, 0.74442554, 0.72919828],
       [0.73354727, 0.75287962, 0.7241143 ],
       [0.73433924, 0.74929017, 0.83233702],
       [0.76254344, 0.75875998, 0.77559745],
       [0.73451585, 0.77248311, 0.79197532],
       [0.74167603, 0.77837813, 0.76843727],
       [0.70716739, 0.72476864, 0.71955955],
       [0.7083602 , 0.72318393, 0.71659815],
       [0.74912465, 0.76287735, 0.77348232],
       [0.73211652, 0.74619669, 0.76579893],
       [0.

In [153]:
mol.calc.get_forces_variance(mol)

/misc/home/qklmn/repos/rotor-gp/code/../../fande/fande/predict/predictors_ase.py:571: UserWarning:

Calculating invariants...



Total number of frames is 1
Total number of batches is 1
Total length of traj is 1
Total number of batches 1
Calculating invariants on trajectory with librascal...


100%|██████████| 1/1 [00:00<00:00,  2.68it/s]
/misc/home/qklmn/repos/rotor-gp/code/../../fande/fande/predict/predictors_ase.py:595: UserWarning:

Predicting...



array([[0.28246722, 0.2569364 , 0.17697105],
       [0.16083393, 0.22986922, 0.22655949],
       [0.26595339, 0.19469914, 0.15795645],
       [0.21528253, 0.12407764, 0.21884617],
       [0.59140593, 0.39451155, 0.44077215],
       [0.43508896, 0.30540356, 0.42349371],
       [0.74045306, 0.78981966, 0.77597141],
       [0.73075187, 0.74705845, 0.75709736],
       [0.74937946, 0.74908561, 0.77944779],
       [0.75806224, 0.73596281, 0.72551358],
       [0.73399311, 0.75456107, 0.74840587],
       [0.71749824, 0.728517  , 0.7514413 ],
       [0.73656768, 0.74442554, 0.72919828],
       [0.73354727, 0.75287962, 0.7241143 ],
       [0.73433924, 0.74929017, 0.83233702],
       [0.76254344, 0.75875998, 0.77559745],
       [0.73451585, 0.77248311, 0.79197532],
       [0.74167603, 0.77837813, 0.76843727],
       [0.70716739, 0.72476864, 0.71955955],
       [0.7083602 , 0.72318393, 0.71659815],
       [0.74912465, 0.76287735, 0.77348232],
       [0.73211652, 0.74619669, 0.76579893],
       [0.

## Compute MD with ML model

In [51]:
### MD with fande calc
print('Testing performance with (meta-)dynamics run...')

from fande.ase import FandeCalc
from ase.units import Bohr,Rydberg,kJ,kB,fs,Hartree,mol,kcal

from ase import io


# from ase.geometry.analysis import Analysis
from ase.constraints import FixAtoms, FixBondLengths
from ase.optimize import BFGS
from ase import units
from ase.io import read
import logging
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.md.verlet import VelocityVerlet
from ase.md.langevin import Langevin
from ase.md.npt import NPT
from ase.md.nptberendsen import NPTBerendsen
from ase.md.nvtberendsen import NVTBerendsen


logging.getLogger("pytorch_lightning").setLevel(logging.ERROR) # logging.ERROR to disable or INFO

data_folder = "/home/qklmn/data/"
traj_test = io.read(data_folder + "datasets/rotors/different_temperatures/300/OUTCAR", format="vasp-out", index = "300:400")

# traj_md = read('../results/test/machine_learning/dftb_opt_1000_six_rings.traj', index=":")
# traj_opt = read('../results/test/machine_learning/opt.traj', index=":")

# atoms = fdm.mol_traj[10].copy()
# atoms = traj_md[300].copy()
# atoms = traj_opt[-1].copy()
atoms = traj_test[10].copy()
atoms.set_pbc(True)


# moving_atoms = sum(atomic_groups, []) 
# fixed_atoms = list( set(range(264)) - set(moving_atoms) )
# fix_atoms = FixAtoms(indices=fixed_atoms)
# atoms.set_constraint(fix_atoms)


# atoms.calc = FandeCalc(predictor)
atoms.calc = fande_calc


# atoms.calc.set_atomic_groups([rings_carbons, rings_hydrogens], titles=["Rings carbons", "Rings hydrogens"])
# atoms.calc.set_forces_errors_plot_file("../results/test/md_runs/forces_errors.png", loginterval=1)
# atoms.calc = LennardJones()

os.makedirs("md_run/", exist_ok=True)

# Verlet dynamics:
# MaxwellBoltzmannDistribution(atoms, temperature_K=300)
# dyn = VelocityVerlet(
#     atoms,
#     dt = 0.1*units.fs,
#     trajectory="md_run/md_test.traj",
#     logfile="md_run/md_log.log",
# )

# dyn = NPT(
#     atoms,
#     # dt = 0.5*units.fs,
#     timestep=0.1,
#     temperature_K=300,
#     externalstress=0.0,
#     trajectory="../results/test/md_runs/md_test.traj",
#     logfile="../results/test/md_runs/md_log.log",
# )

# dyn = NPTBerendsen(atoms, timestep=0.1 * units.fs, temperature_K=300,
#                    taut=100 * units.fs, pressure_au=1.01325 * units.bar,
#                    taup=1000 * units.fs, compressibility=4.57e-5 / units.bar,
#                    trajectory="../results/test/md_runs/md_test.traj",
#                    logfile="../results/test/md_runs/md_log.log",)

# import os

# MaxwellBoltzmannDistribution(atoms, temperature_K=300, force_temp=True)
# dyn = NVTBerendsen(atoms, 0.2 * units.fs, 300, taut=0.5*1000*units.fs, 
#                    trajectory="md_run/md_test.traj",   
#                    logfile="md_run/md_log.log")

# dyn.run(100)

# Langevin dynamics:
# https://databases.fysik.dtu.dk/ase/tutorials/md/md.html
# MaxwellBoltzmannDistribution(atoms, temperature_K=100, force_temp=True)
dyn = Langevin(atoms, 0.2*fs, 
               temperature_K=300, #0.1/units.kB, 
               friction=0.1,
        #        fixcm=True, 
               trajectory='md_run/md_test.traj',
               logfile="md_run/md_log.log")

dyn.run(10)

# # Structure optimization:
# dyn = BFGS(
#     atoms,
#     trajectory="../results/test/md_runs/md_test.traj",
#     logfile="../results/test/md_runs/md_log.log",)
# dyn.run(fmax=0.1)


print(" ALL JOBS WITHIN PYTHON SCRIPT ARE DONE! ")

print("TIMING: ", time.time()-start_time, " seconds")

Testing performance with (meta-)dynamics run...


/misc/home/qklmn/repos/rotor-gp/code/../../fande/fande/predict/predictors_ase.py:571: UserWarning:

Calculating invariants...



Total number of frames is 1
Total number of batches is 1
Total length of traj is 1
Total number of batches 1
Calculating invariants on trajectory with librascal...


100%|██████████| 1/1 [00:00<00:00,  5.10it/s]
/misc/home/qklmn/repos/rotor-gp/code/../../fande/fande/predict/predictors_ase.py:595: UserWarning:

Predicting...



Total number of frames is 1
Total number of batches is 1
Total length of traj is 1
Total number of batches 1
Calculating invariants on trajectory with librascal...


...

100%|██████████| 1/1 [00:00<00:00,  5.18it/s]


Total number of frames is 1
Total number of batches is 1
Total length of traj is 1
Total number of batches 1
Calculating invariants on trajectory with librascal...


100%|██████████| 1/1 [00:00<00:00,  5.20it/s]


 ALL JOBS WITHIN PYTHON SCRIPT ARE DONE! 


NameError: name 'time' is not defined

In [ ]:
# make a nice starting configuration for i-pi
from ase import io

data_folder = "/home/qklmn/data/"
atoms = read(data_folder + "datasets/rotors/different_temperatures/300/OUTCAR", format="vasp-out", index = "350")
# io.write("/home/qklmn/data/starting_configuration/1.cif", atoms, format='cif')

# Make dftb+ trajectory:

In [121]:
from ase import io
# from ase.geometry.analysis import Analysis
from ase.constraints import FixAtoms, FixBondLengths
from ase.optimize import BFGS
from ase import units
from ase.io import read
import logging
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.md.verlet import VelocityVerlet
from ase.md.langevin import Langevin
from ase.md.npt import NPT
from ase.md.nptberendsen import NPTBerendsen
from ase.md.nvtberendsen import NVTBerendsen

from ase.units import Bohr,Rydberg,kJ,kB,fs,Hartree,mol,kcal



from ase.calculators.dftb import Dftb
import os
os.environ['OMP_NUM_THREADS'] = "6,1"
os.environ["ASE_DFTB_COMMAND"] = "ulimit -s unlimited; /usr/local/dftbplus-21.2/bin/dftb+ > PREFIX.out"
# os.environ["ASE_DFTB_COMMAND"] = "dftb+ > PREFIX.out"
os.environ["DFTB_PREFIX"] = "/home/qklmn/data/dftb/pbc-0-3"




# initial_structure = io.read("/home/qklmn/data/starting_configuration/1.cif")
crystal_264=io.read("/home/qklmn/data/starting_configuration/1.cif")





logging.getLogger("pytorch_lightning").setLevel(logging.ERROR) # logging.ERROR to disable or INFO

data_folder = "/home/qklmn/data/"
traj_test = io.read(data_folder + "datasets/rotors/different_temperatures/300/OUTCAR", format="vasp-out", index = "300:400")

# traj_md = read('../results/test/machine_learning/dftb_opt_1000_six_rings.traj', index=":")
# traj_opt = read('../results/test/machine_learning/opt.traj', index=":")

# atoms = fdm.mol_traj[10].copy()
# atoms = traj_md[300].copy()
# atoms = traj_opt[-1].copy()
# atoms = traj_test[10].copy()

atoms = crystal_264.copy()
atoms.set_pbc(True)


calc = Dftb(atoms=atoms,
        label='crystal',
        # Hamiltonian_ = "xTB",
        # # Hamiltonian_Method = "GFN1-xTB",
        # Hamiltonian_MaxAngularMomentum_='',
        # Hamiltonian_MaxAngularMomentum_O='p',
        # Hamiltonian_MaxAngularMomentum_H='s',
        # Hamiltonian_MaxAngularMomentum_N='s',
        # Hamiltonian_MaxAngularMomentum_C='s',
        # Hamiltonian_MaxAngularMomentum_Si='s',
        kpts=(1,1,1),
        # Hamiltonian_SCC='Yes',
        # Verbosity=0,
        # Hamiltonian_OrbitalResolvedSCC = 'Yes',
        # Hamiltonian_SCCTolerance=1e-15,
        # kpts=None,
        # Driver_='ConjugateGradient',
        # Driver_MaxForceComponent=1e-3,
        # Driver_MaxSteps=200,
        # Driver_LatticeOpt = 'Yes',
        #     Driver_AppendGeometries = 'Yes',
        #     Driver_='',
        #     Driver_Socket_='',
        #     Driver_Socket_File='Hello'
        )

atoms.calc = calc


os.chdir("/home/qklmn/data/datasets/rotors/different_temperatures/dftb")



# Langevin dynamics:
# https://databases.fysik.dtu.dk/ase/tutorials/md/md.html
# MaxwellBoltzmannDistribution(atoms, temperature_K=100, force_temp=True)
dyn = Langevin(atoms, 1*fs, 
               temperature_K=300, #0.1/units.kB, 
               friction=0.1,
        #        fixcm=True, 
               trajectory='traj_300.traj',
               logfile="md_log.log")

dyn.run(1000)

True

In [217]:
import matplotlib.pyplot as plt

traj = io.read("/home/qklmn/data/datasets/rotors/different_temperatures/dftb/traj_2100.traj", index=":")

energies = [mol.get_temperature() for mol in traj]
# plt.plot(energies)
# plt.show()

In [122]:
traj_dftb_2100 = io.read("/home/qklmn/data/datasets/rotors/different_temperatures/dftb/traj_2100.traj", index = ":")

In [123]:
initial_structure = io.read("/home/qklmn/data/starting_configuration/1.cif")

In [218]:
# print(*traj_dftb_2100[0].get_chemical_symbols() )
# print(*initial_structure.get_chemical_symbols() )

False